In [1]:
import numpy as np
import torch
import tntorch as tn

#PARAMETRI GAUSSIANA

num_dimensions = 1
mu = 0.10   # Mean vector
cov_matrix = 0.20  # Covariance matrix 

#PARAMETRI DISCRETIZZAZIOINE

d = 10                 # qubit
m = 2**d               # dimensioni discretizzazione

#GAUSSIANA DISCRETA

domain_np = np.linspace(mu - 3*np.sqrt(cov_matrix), mu + 3*np.sqrt(cov_matrix), m)
def gaussian(x):
    return 1/(np.sqrt(2*np.pi*cov_matrix)) * np.exp(-0.5*((x-mu)**2)/np.sqrt(cov_matrix))


vec =  np.array([gaussian(x) for x in domain_np])    # vettore probabilità discreta

shape = (2,)*d         # (2,2,2,2)
A = vec.reshape(shape) #tensore numpy
T=tn.Tensor(A)      #tensore torch


#CREAZIONE TENSOR TRAIN

TTrain = tn.cross(
    function=lambda x: x,   # identità su ciascuna fibra
    tensors=[T],            # lista di un solo tensore               # tolleranza desiderata
    ranks_tt=8,                 # rank massimo ammesso
)



print(TTrain)


cores_torch = TTrain.cores
cores = [c.cpu().numpy() for c in cores_torch]

print(len(cores))
print(cores[0])
print(cores[2].shape)

In [2]:
W = []
nk = []
for k in range (len(cores)):
     nk.append(len(cores[k-1][0,0,:]))
print(nk)

for k in range(len(cores)):
    # reshape & SVD 
    j, i, n_k = cores[k].shape
    
    cores[k] = np.reshape(cores[k], (j * i, n_k))

    #print(cores[k].shape)
    U, S, V = np.linalg.svd(cores[k])
    S_prime = np.zeros_like(cores[k])
    for i in range(len(S)):
        S_prime[i,i] = S[i]

    R = S_prime @ V

    tronc = min(2**k,nk[k])
    R = R[:tronc, :]

    if k != len(cores) - 1:
        cores[k+1] = np.tensordot(R, cores[k+1],axes=([-1], [0]))

    # calcolo dei qubit coinvolti
    start = k + 1
    mn    = min(start, int(np.log2(nk[k])))
    diff  = int(start - mn)
    qubits = list(range(diff-1, start))
    print(U.shape)
    U_list = U.tolist()
    W.append([U_list, qubits])

# Stampa di controllo
for idx, (U_list, qubits) in enumerate(W):
    print(f"Unitary {idx}: acts on qubits {qubits}, matrix with dimension {len(U_list)}")

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.library import UnitaryGate
from qiskit.visualization import plot_histogram

n_qubits = 10
gates = W[::-1]   # or wherever you've defined W

qr = QuantumRegister(n_qubits, "q")
cr = ClassicalRegister(n_qubits, "c")
qc = QuantumCircuit(qr, cr)

for matrix, idxs in gates:
    # wrap your numpy array as a UnitaryGate
    ug = UnitaryGate(matrix)
    # map integer indices → Qubit objects
    qubit_list = [qr[i] for i in idxs ]
    qc.append(ug, qubit_list)

qc.draw('mpl')
# measure *all* qubits
qc.measure(qr, cr)

# simulate
sim = AerSimulator()
t_qc = transpile(qc, sim, optimization_level=0)
job = sim.run(t_qc, shots=10_000)
counts = job.result().get_counts()

# plot
plot_histogram(counts)

In [2]:
W = []
nk = []
for k in range (len(cores)):
     nk.append(len(cores[k-1][0,0,:]))
print(nk)

for k in range(len(cores)):
    # reshape & SVD 
    j, i, n_k = cores[k].shape
    
    cores[k] = np.reshape(cores[k], (j * i, n_k))

    #print(cores[k].shape)
    U, S, V = np.linalg.svd(cores[k])
    S_prime = np.zeros_like(cores[k])
    for i in range(len(S)):
        S_prime[i,i] = S[i]

    R = S_prime @ V

    tronc = min(2**k,nk[k])
    R = R[:tronc, :]

    if k != len(cores) - 1:
        cores[k+1] = np.tensordot(R, cores[k+1],axes=([-1], [0]))

    # calcolo dei qubit coinvolti
    start = k + 1
    mn    = min(start, int(np.log2(nk[k])))
    diff  = int(start - mn)
    qubits = list(range(diff-1, start))
    print(U.shape)
    U_list = U.tolist()
    W.append([U_list, qubits])

# Stampa di controllo
for idx, (U_list, qubits) in enumerate(W):
    print(f"Unitary {idx}: acts on qubits {qubits}, matrix with dimension {len(U_list)}")

In [2]:
W = []
nk = []
for k in range (len(cores)):
     nk.append(len(cores[k-1][0,0,:]))
print(nk)

for k in range(len(cores)):
    # reshape & SVD 
    j, i, n_k = cores[k].shape
    
    cores[k] = np.reshape(cores[k], (j * i, n_k))

    #print(cores[k].shape)
    U, S, V = np.linalg.svd(cores[k])
    S_prime = np.zeros_like(cores[k])
    for i in range(len(S)):
        S_prime[i,i] = S[i]

    R = S_prime @ V

    tronc = min(2**k,nk[k])
    R = R[:tronc, :]

    if k != len(cores) - 1:
        cores[k+1] = np.tensordot(R, cores[k+1],axes=([-1], [0]))

    # calcolo dei qubit coinvolti
    start = k + 1
    mn    = min(start, int(np.log2(nk[k])))
    diff  = int(start - mn)
    qubits = list(range(diff-1, start))
    print(U.shape)
    U_list = U.tolist()
    W.append([U_list, qubits])

# Stampa di controllo
for idx, (U_list, qubits) in enumerate(W):
    print(f"Unitary {idx}: acts on qubits {qubits}, matrix with dimension {len(U_list)}")

In [2]:
W = []
nk = []
for k in range (len(cores)):
     nk.append(len(cores[k-1][0,0,:]))
print(nk)

for k in range(len(cores)):
    # reshape & SVD 
    j, i, n_k = cores[k].shape
    
    cores[k] = np.reshape(cores[k], (j * i, n_k))

    #print(cores[k].shape)
    U, S, V = np.linalg.svd(cores[k])
    S_prime = np.zeros_like(cores[k])
    for i in range(len(S)):
        S_prime[i,i] = S[i]

    R = S_prime @ V

    tronc = min(2**k,nk[k])
    R = R[:tronc, :]

    if k != len(cores) - 1:
        cores[k+1] = np.tensordot(R, cores[k+1],axes=([-1], [0]))

    # calcolo dei qubit coinvolti
    start = k + 1
    mn    = min(start, int(np.log2(nk[k])))
    diff  = int(start - mn)
    qubits = list(range(diff-1, start))
    print(U.shape)
    U_list = U.tolist()
    W.append([U_list, qubits])

# Stampa di controllo
for idx, (U_list, qubits) in enumerate(W):
    print(f"Unitary {idx}: acts on qubits {qubits}, matrix with dimension {len(U_list)}")

[1, 2, 4, 8, 8, 8, 8, 8, 4, 2]
(2, 2)
(2, 2)
(4, 4)
(8, 8)
(16, 16)
(16, 16)
(16, 16)
(16, 16)
(16, 16)
(8, 8)
Unitary 0: acts on qubits [0], matrix with dimension 2
Unitary 1: acts on qubits [0, 1], matrix with dimension 2
Unitary 2: acts on qubits [0, 1, 2], matrix with dimension 4
Unitary 3: acts on qubits [0, 1, 2, 3], matrix with dimension 8
Unitary 4: acts on qubits [1, 2, 3, 4], matrix with dimension 16
Unitary 5: acts on qubits [2, 3, 4, 5], matrix with dimension 16
Unitary 6: acts on qubits [3, 4, 5, 6], matrix with dimension 16
Unitary 7: acts on qubits [4, 5, 6, 7], matrix with dimension 16
Unitary 8: acts on qubits [6, 7, 8], matrix with dimension 16
Unitary 9: acts on qubits [8, 9], matrix with dimension 8
